<a href="https://colab.research.google.com/github/Ragavan1303/EinsteinFieldEquations/blob/main/5_Raster_Clipped.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install rasterio geopandas shapely fiona matplotlib

import os
import zipfile
import rasterio
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from rasterio.mask import mask
from google.colab import files
import fiona
import rasterio
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from rasterio.mask import mask
import os
import shutil



In [ ]:
def upload_files():
  uploaded = files.upload()
  return list(uploaded.keys())

# Step 1: Upload the raster files
print("Upload your five raster (.tif) files:")
raster_files = upload_files()
if len(raster_files) != 5:
    raise ValueError("Please upload exactly five raster files.")

# Step 2: Upload the shapefile (as a ZIP)
print("Upload your shapefile (.zip) file:")
shapefile_zip = upload_files()[0]

# Check if the uploaded file is a zip file
if not shapefile_zip.lower().endswith(".zip"):
    raise ValueError("Uploaded shapefile is not a ZIP file.")

# Step 3: Extract the shapefile
shapefile_dir = "shapefile"
os.makedirs(shapefile_dir, exist_ok=True)
with zipfile.ZipFile(shapefile_zip, 'r') as zip_ref:
    zip_ref.extractall(shapefile_dir)

# Recursively search for .shp file inside the extracted folder
shapefile_path = None
for root, _, files in os.walk(shapefile_dir):
    for file in files:
        if file.endswith(".shp"):
            shapefile_path = os.path.join(root, file)
            break
    if shapefile_path:
        break
if shapefile_path is None:
    raise FileNotFoundError("No .shp file found in the uploaded ZIP.")

# Step 4: Process each raster file
for raster_path in raster_files:
    try:
        # Read the shapefile
        shapefile = gpd.read_file(shapefile_path)

        # Read the raster data
        with rasterio.open(raster_path) as src:
            out_image, out_transform = mask(src, shapefile.geometry, crop=True)
            out_meta = src.meta.copy()

        # Update metadata
        out_meta.update({"driver": "GTiff",
                         "height": out_image.shape[1],
                         "width": out_image.shape[2],
                         "transform": out_transform})

        # Define output filename
        output_filename = os.path.splitext(raster_path)[0] + "_clipped.tif"

        # Write the clipped raster
        with rasterio.open(output_filename, "w", **out_meta) as dest:
            dest.write(out_image)
        print(f"Clipped raster saved as: {output_filename}")

    except Exception as e:
        print(f"Error processing {raster_path}: {e}")
